In [8]:
import requests
import pandas as pd
import os
import dotenv
dotenv.load_dotenv()


year = 2023
league_id = os.getenv('league_id')
espn_s2 = os.getenv('espn_s2')
swid = os.getenv('swid')

In [9]:
cookies = {
    'espn_s2': espn_s2,
    'SWID': swid
}

# Ensure we're using the correct endpoint and parameters to fetch the needed data
url = f"https://fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}"
params = {
    "view": "mMatchupScore"  # This view should give us the matchups and scores
}

response = requests.get(url, cookies=cookies, params=params)
data = response.json()

In [10]:
# Assuming 'schedule' is the correct key for matchups based on the API's usual structure
matchups = data.get('schedule', [])

# We'll also need to fetch team details for names and owners, possibly with a separate request or adjusted parameters
team_info_url = f"{url}?view=mTeam"
team_response = requests.get(team_info_url, cookies=cookies)
teams_data = team_response.json()

# print(teams_data.get('teams',[]))

teams = {team['id']: {'name': team.get('name', '')
                      } for team in teams_data.get('teams', [])}

# print(teams)

In [11]:
# Initialize a list for detailed matchup data
detailed_data = []

for week in matchups:
    week_num = week.get('matchupPeriodId')
    for matchup in ['home', 'away']:
        team = week.get(matchup)
        if team:
            team_id = team.get('teamId')
            score = team.get('totalPoints')
            team_name = teams.get(team_id, {}).get('name', 'Unknown Team')
            # owner = teams.get(team_id, {}).get('owner', 'Unknown Owner')

            detailed_data.append({
                'Week': week_num,
                'Team ID': team_id,
                'Score': score,
                'Team Name': team_name,
                # 'Owner': owner
            })

# Convert to DataFrame
df = pd.DataFrame(detailed_data)

In [12]:
# df

In [13]:
# Find highest scoring team for each week
highest_scores_per_week = df.sort_values('Score', ascending=False).drop_duplicates(['Week'])
# Now sort the resulting DataFrame in ascending order by 'Week'
highest_scores_per_week = highest_scores_per_week.sort_values('Week', ascending=True)

In [14]:
hs_df = highest_scores_per_week[['Week', 'Team Name', 'Score']].set_index('Week')
hs_df.to_csv('outputs/weekly_high_scores_' + str(year) + '.csv')

print(hs_df)

     Week               Team Name   Score
6       1       Bloodbath Mcgrath  131.56
21      2       Bloodbath Mcgrath  159.26
24      3             lil' Scamps  151.56
47      4             Taylor Made  150.10
59      5             Taylor Made  151.58
64      6             BEC Michael  125.66
82      7               EL Diablo  149.26
85      8             BEC Michael  126.40
98      9             BEC Michael  139.50
119    10           Naturally Lit  134.48
130    11            BDE Culture   135.30
141    12             BEC Michael  133.06
154    13             BEC Michael  143.96
157    14             lil' Scamps  142.44
170    15  Make Gronk Great Again  146.06
181    16  Make Gronk Great Again  136.38
195    17             Taylor Made  166.22
